In [128]:
from altair import Chart, Color, Tooltip, X, Y
from pandas import DataFrame

from app.data import MongoDB


In [112]:
def pie_chart(df, column, title):
    chart = Chart(
        df,
        title=title,
    ).mark_arc(innerRadius=80).encode(
        color=Color(column),
        theta="count()",
        tooltip=Tooltip([column, "count()"]),
    ).properties(
        width=480,
        height=400,
        padding=24,
    ).configure(
        legend={"padding": 24},
        title={"fontSize": 20, "offset": 24},
        view={"stroke": "#FFF"},
    )
    return chart

In [113]:
DB = MongoDB()


In [114]:
mentors = DB.projection("Mentors", {
    "validate_status": "approved",
    "is_active": True,
    "accepting_new_mentees": True,
}, {"tech_stack": True})


In [115]:
mentors = DataFrame(mentors)


In [116]:
mentors = mentors.explode(ignore_index=True, column="tech_stack")


In [149]:
pie_chart(mentors, "tech_stack", "Mentor Tech Stacks")


alt.Chart(...)

In [118]:
mentees = DB.projection("Mentees", {
    "is_active": True,
    "validate_status": "approved",
}, {"tech_stack": True})


In [119]:
mentees = DataFrame(mentees)


In [120]:
pie_chart(mentees, "tech_stack", "Mentee Tech Stacks")


alt.Chart(...)

### Stacked Bar Chart

In [143]:
import altair as alt
from vega_datasets import data


In [144]:
source = data.barley()
source.head()

,yield,variety,year,site
0,27.00000,Manchuria,1931,University Farm
1,48.86667,Manchuria,1931,Waseca
2,27.43334,Manchuria,1931,Morris
3,39.93333,Manchuria,1931,Crookston
4,32.96667,Manchuria,1931,Grand Rapids


In [148]:
alt.Chart(source, title="yield by variety for site").mark_bar().encode(
    x='variety',    # Tech Stacks
    y='sum(yield)', # Count Tech Stacks
    color='site',   # Role
)

alt.Chart(...)

---

In [156]:
mentors = DB.projection("Mentors", {
    "validate_status": "approved",
    "is_active": True,
    "accepting_new_mentees": True,
}, {"tech_stack": True})
mentors_df = DataFrame(mentors)
mentors_df

,tech_stack
0,"[Backend, Design UI/UX]"
1,[iOS]
2,[Frontend]
3,"[Backend, Career Development, Android]"
4,"[Backend, Design UI/UX]"
5,"[Android, iOS]"
6,[Backend]
7,[Frontend]
8,"[Frontend, Backend, Design UI/UX]"
9,[Backend]


In [157]:
mentees = DB.projection("Mentees", {
    "is_active": True,
    "validate_status": "approved",
}, {"tech_stack": True})
mentees_df = DataFrame(mentees)
mentees_df

,tech_stack
0,Android
1,Design UI/UX
2,Android
3,Android
4,Android
...,...
92,Android
93,Android
94,Design UI/UX
95,Data Science
